In [ ]:
#### ---------
#### use scanpy.yml environment
#### ----------

import os 
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.gridspec

In [ ]:
# define cell type colors for consistency across plots 
cell_type_colors = {
    'Epithelial' : '#d60000', 
    'Basal Epithelial' : '#8c3bff', 
    'Proliferative Epithelial' : '#018700',
    'Proliferative Basal Epithelial' : '#00acc6', 
    'Myoepithelial' : '#97ff00', 
    'Stroma' : '#ff7ed1', 
    'Endothelial' : '#6b004f',
    'aSMA+ Stroma' : '#ffa52f', 
    'B-cells' : '#00009c',
    'T-cells' : '#857067', 
    'CD8+ T-cells' : '#004942', 
    'CD4+ T-cells' : '#4f2a00',
    'Regulatory T-cells' : '#00fdcf', 
    'Macrophages' : '#bcb6ff',
    'CD163+ Macrophages' : '#95b379'   
}

In [ ]:
# read data 
df = pd.read_csv('data/E06_tracked_indiv_leiden.csv')

# read annotations and turn into a dictionary with leiden clusters as keys 
annotation_df = pd.read_csv(f'annotations/E06_tracked_annotations.csv')
annotation_map = pd.Series(annotation_df.cell_type_refined.values, index = annotation_df.cluster).to_dict()

In [ ]:
# columns to subset by: lineage markers from both panels and leiden column
cols = ['tumor_CD3', 'tumor_Vimentin', 'tumor_aSMA',
 'tumor_Ecad', 'tumor_Ki67', 'tumor_CD45', 'tumor_CK14',
 'tumor_CK19', 'tumor_CK17', 'tumor_PCNA', 
 'tumor_PanCK', 'tumor_CD31', 'immune_CD3', 
 'immune_GRZB', 'immune_Ki67', 'immune_PanCK', 
 'immune_CD45', 'immune_CD68', 'immune_CD3d', 
 'immune_CD8a', 'immune_CD163','immune_aSMA', 
 'immune_CD14', 'immune_CD4', 'immune_FOXP3',
 'immune_CD11b','immune_CD20','immune_MHC_II_DPB1',
 'leiden']

In [ ]:
# calculate mean marker intensity by leiden cluster and map the cell type annotations 
df = df[cols]
df = df.groupby('leiden').mean()
obs = df.reset_index()[['leiden']]
obs['cell_type_tracked'] = obs['leiden'].map(annotation_map)

# map colors to cell types 
obs['Cell Type'] = obs['cell_type_tracked'].map(cell_type_colors)

In [ ]:
# clean up a couple marker names 
df.columns = ['tumor_CD3', 'tumor_Vimentin', 'tumor_aSMA', 'tumor_Ecad', 'tumor_Ki67',
       'tumor_CD45', 'tumor_CK14', 'tumor_CK19', 'tumor_CK17', 'tumor_PCNA',
       'tumor_PanCK', 'tumor_CD31', 'immune_CD3', 'immune_GRZB', 'immune_Ki67',
       'immune_PanCK', 'immune_CD45', 'immune_CD68', 'immune_CD3d',
       'immune_CD8a', 'immune_CD163', 'immune_aSMA', 'immune_CD14',
       'immune_CD4', 'immune_FOXP3', 'immune_CD11b', 'immune_CD20',
       'immune_MHCII-DPB1']

In [ ]:
## --------------------- main clustermap 
g = sns.clustermap(
    df, 
    z_score=1, 
    cmap = 'Spectral_r', 
    vmin = -2, 
    vmax = 2,
    row_colors = obs['Cell Type'],
    figsize=(8,8))

g.fig.set_facecolor('white')

# make room for legends/colorbar 
g.gs.update(left=0.01, right=0.8)

## --------------------- update axis labels to show panel by fontface 
ax = g.ax_heatmap
labels = ax.get_xticklabels()
for label in labels:
    if 'tumor_' in label.get_text():
        label.set_fontweight('bold')
    
    label.set_text(re.split('_', label.get_text())[-1])

ax.set_xticklabels(labels)
ax.set_ylabel('Cluster',size = 12)

## --------------------- create a legend for panel marker specification 
text = r"$\bf{Tumor}$" + " " + r"$\bf{Panel}$" + "\n" + "Immune Panel"
props = dict(facecolor='white', alpha=1)
# ax.text(1, 1, text, fontsize = 11, verticalalignment='top', bbox=props)
plt.gcf().text(0.85, 0.14, text, fontsize=11, bbox=props)

## --------------------- move the colorbar 
# g.cax.set_visible(False)
g.ax_cbar.set_aspect(1)
cax = g.ax_cbar
box = cax.get_position()
box.x0 = box.x0 + 0.8
box.x1 = box.x1 + 0.8
box.y0 = box.y0 - 0.6
box.y1 = box.y1 - 0.6
box = cax.set_position(box)
g.ax_cbar.set_ylabel("z-score intensity",size=12)

plt.savefig('Figure_4C.png', dpi = 500)